In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import matplotlib.pyplot as plt
import numpy as np
import copy
import os
import random
import shutil
import time
from PIL import Image
from torch.autograd import Variable

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


ROOT = "/content/drive/MyDrive"

images_dir = os.path.join(ROOT, 'images')
train_labeled_dir = os.path.join(images_dir, 'training_labeled_images')
test_labeled_dir = os.path.join(images_dir, 'testing_labeled_images')


# Calculate means, stds using  in transforms as parameter

In [ ]:
BATCH_SIZE = 32
train_data = datasets.ImageFolder(root=train_labeled_dir,
                                  transform=transforms.ToTensor())
# calculate mean and std which is used in transforms parameter.
means = torch.zeros(3)
stds = torch.zeros(3)

for img, label in train_data:
    means += torch.mean(img, dim=(1, 2))
    stds += torch.std(img, dim=(1, 2))
# setting image to fit pretrained model.
pretrained_size = 224
means /= len(train_data)
stds /= len(train_data)

print(f'Calculated means: {means}')
print(f'Calculated stds: {stds}')


# data augmentation, data normalization, and load data

In [ ]:
train_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.RandomRotation(5),
                           transforms.RandomHorizontalFlip(0.5),
                           transforms.RandomCrop(pretrained_size, padding=10),
                           transforms.ToTensor(),
                           transforms.Normalize(mean=means, std=stds)
                       ])

test_transforms = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.CenterCrop(pretrained_size),
                           transforms.ToTensor(),
                           transforms.Normalize(mean=means, std=stds),
                       ])

train_data = datasets.ImageFolder(root=train_labeled_dir,
                                  transform=train_transforms)
test_data = datasets.ImageFolder(root=test_labeled_dir,
                                 transform=test_transforms)

train_iterator = data.DataLoader(train_data,
                                 batch_size=BATCH_SIZE,
                                 shuffle=True)
test_iterator = data.DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=False)
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')


cuda
Calculated means: tensor([0.4819, 0.4976, 0.4321])
Calculated stds: tensor([0.1878, 0.1864, 0.1992])
Number of training examples: 3000
Number of testing examples: 3033


# Setting the parameter of pretrained model

In [ ]:
pretrained_model = models.resnet152(pretrained=True)
# setting the input size and output size to fit the pretrained model
# and the 200 classes of bird images.
IN_FEATURES = pretrained_model.fc.in_features
OUTPUT_DIM = len(test_data.classes)
# print(OUTPUT_DIM)
pretrained_model.fc = nn.Linear(IN_FEATURES, OUTPUT_DIM)
pretrained_model = pretrained_model.to(device)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = optim.SGD(pretrained_model.parameters(), lr=0.001, momentum=0.9)

lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max',
                                                   patience=3, threshold=0.9)


# Function to training and testing

In [ ]:
def train_model(model, criterion, optimizer, scheduler, n_epochs):
    losses = []
    accuracies = []
    test_accuracies = []
    # set the model to train mode initially
    model.train()
    for epoch in range(n_epochs):
        since = time.time()
        running_loss = 0.0
        running_correct = 0.0
        for i, data in enumerate(train_iterator, 0):
            # get the inputs and assign them to cuda
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            # print((outputs))
            # print((labels))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # calculate the loss/acc later
            running_loss += loss.item()
            running_correct += (labels == predicted).sum().item()
        epoch_duration = time.time()-since
        epoch_loss = running_loss/len(train_iterator)
        epoch_acc = 100/BATCH_SIZE*running_correct/len(train_iterator)
        print("Epoch %s, duration: %d s, loss: %.4f, acc: %.4f "
              % (epoch+1, epoch_duration, epoch_loss, epoch_acc))
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        # switch the model to eval mode to evaluate on test data
        model.eval()
        test_acc = eval_model(model)
        test_accuracies.append(test_acc)
        # set the model to train mode after validating
        model.train()
        scheduler.step(test_acc)
        since = time.time()
    print('Finished Training')
    return model, losses, accuracies, test_accuracies


def eval_model(model):
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for i, data in enumerate(test_iterator, 0):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_acc = 100.0 * correct / total
    print('Accuracy of the network on the test images: %d %%' % (
        test_acc))
    return test_acc


# Starting to train the model

In [ ]:
pretrained_model, training_losses, training_accs, test_accs =
train_model(pretrained_model, criterion, optimizer, lrscheduler, n_epochs=10)


Epoch 1, duration: 184 s, loss: 5.2241, acc: 2.6263 
Accuracy of the network on the test images: 9 %
Epoch 2, duration: 184 s, loss: 4.6762, acc: 18.3511 
Accuracy of the network on the test images: 23 %
Epoch 3, duration: 184 s, loss: 4.0103, acc: 34.7739 
Accuracy of the network on the test images: 29 %
Epoch 4, duration: 184 s, loss: 3.3874, acc: 45.1463 
Accuracy of the network on the test images: 37 %
Epoch 5, duration: 184 s, loss: 2.8796, acc: 53.9229 
Accuracy of the network on the test images: 42 %
Epoch 6, duration: 184 s, loss: 2.4061, acc: 62.7660 
Accuracy of the network on the test images: 45 %
Epoch 7, duration: 183 s, loss: 2.0279, acc: 70.0133 
Accuracy of the network on the test images: 48 %
Epoch 8, duration: 183 s, loss: 1.7130, acc: 75.9641 
Accuracy of the network on the test images: 50 %
Epoch 9, duration: 184 s, loss: 1.4475, acc: 79.9202 
Accuracy of the network on the test images: 51 %
Epoch 10, duration: 184 s, loss: 1.2119, acc: 83.6769 
Accuracy of the netw

# Because of the training accuracy is not very high, training for more epochs

In [ ]:
pretrained_model, training_losses, training_accs, test_accs =
train_model(pretrained_model, criterion, optimizer, lrscheduler, n_epochs=5)


Epoch 1, duration: 184 s, loss: 1.0496, acc: 88.2979 
Accuracy of the network on the test images: 53 %
Epoch 2, duration: 184 s, loss: 1.0065, acc: 90.0598 
Accuracy of the network on the test images: 53 %
Epoch 3, duration: 184 s, loss: 0.9934, acc: 90.5253 
Accuracy of the network on the test images: 53 %
Epoch 4, duration: 184 s, loss: 0.9866, acc: 90.1263 
Accuracy of the network on the test images: 54 %
Epoch 5, duration: 184 s, loss: 0.9597, acc: 90.5253 
Accuracy of the network on the test images: 54 %
Finished Training


# Save the trained model

In [ ]:
torch.save({"model_state_dict": pretrained_model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict()},
           "/content/drive/MyDrive/Adjust_resnet152.pth")


# Load the trained model

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/Adjust_resnet152.pth")
pretrained_model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])


# Using the model to predict the test image and save the labels in answer.txt.

In [ ]:
pretrained_model.eval()
path = "/content/drive/MyDrive/images/testing_images"


def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  # this is for VGG, may not be needed for ResNet
    return image  # assumes that you're using GPU

loader = transforms.Compose([
                           transforms.Resize(pretrained_size),
                           transforms.CenterCrop(pretrained_size),
                           transforms.ToTensor(),
                           transforms.Normalize(mean=means, std=stds)
                           ])
dicts = {}
keys = range(200)
with open('/content/drive/MyDrive/classes.txt') as f:
    values = [x.strip() for x in f.readlines()]
    for i in keys:
        dicts[i] = values[i]

print(dict(list(enumerate(values))))


with open('/content/drive/MyDrive/testing_img_order.txt') as f:
    test_images = [x.strip() for x in f.readlines()]  # all the testing images

submission = []
for img in test_images:  # image order is important to your result
    img_path = os.path.join(path, img)
    # print(img_path)
    image = image_loader(img_path).to(device)
    outputs = pretrained_model(image)
    _, predicted = torch.max(outputs.data, 1)
    # print(predicted)
    s = str(predicted)
    s = int(s.split("[")[1].split("]")[0])
    # print(s)
    # print(s.split("[")[1].split("]")[0])
    # predicted_class = s.split("[")[1].split("]")[0]
    submission.append([img, dicts[s]])
    # print([img, dicts[s]])

np.savetxt("/content/drive/MyDrive/answer.txt", submission, fmt='%s')


{0: '001.Black_footed_Albatross', 1: '002.Laysan_Albatross', 2: '003.Sooty_Albatross', 3: '004.Groove_billed_Ani', 4: '005.Crested_Auklet', 5: '006.Least_Auklet', 6: '007.Parakeet_Auklet', 7: '008.Rhinoceros_Auklet', 8: '009.Brewer_Blackbird', 9: '010.Red_winged_Blackbird', 10: '011.Rusty_Blackbird', 11: '012.Yellow_headed_Blackbird', 12: '013.Bobolink', 13: '014.Indigo_Bunting', 14: '015.Lazuli_Bunting', 15: '016.Painted_Bunting', 16: '017.Cardinal', 17: '018.Spotted_Catbird', 18: '019.Gray_Catbird', 19: '020.Yellow_breasted_Chat', 20: '021.Eastern_Towhee', 21: '022.Chuck_will_Widow', 22: '023.Brandt_Cormorant', 23: '024.Red_faced_Cormorant', 24: '025.Pelagic_Cormorant', 25: '026.Bronzed_Cowbird', 26: '027.Shiny_Cowbird', 27: '028.Brown_Creeper', 28: '029.American_Crow', 29: '030.Fish_Crow', 30: '031.Black_billed_Cuckoo', 31: '032.Mangrove_Cuckoo', 32: '033.Yellow_billed_Cuckoo', 33: '034.Gray_crowned_Rosy_Finch', 34: '035.Purple_Finch', 35: '036.Northern_Flicker', 36: '037.Acadian_Fl